In [1]:
import pandas as pd
import numpy as np

import pyomo.environ as pyo

import gurobipy as gp
from gurobipy import GRB

import openpyxl

from icecream import ic


In [2]:
###
### RETRIEVE AND CLEAN DATA
###
file_path = "Railway services-2024.xlsx"

try:
    df = pd.read_excel(io=file_path, engine="openpyxl")
    ic(df.head())

except FileNotFoundError:
    print(f"File '{file_path}' not found. Please check the file path.")

# Data cleaning/ enhancing

# Removing unneccesary spaces from 'From' and 'To' columns. 
df['From'] = df['From'].str.strip()
df['To'] = df['To'].str.strip()

# Adding an indicator when the trip is located on line 400
indicator_line_400 = np.where(df['Line'] == 400.0, 1, 0)
df["line_400"] = indicator_line_400


ic| df.head():    Trip Departure Time Arrival Time From To  Demand(μ)  Demand(σ)  Line
               0     1       07:00:00     07:46:00    M  A        327         64   100
               1     2       07:00:00     07:57:00    H  M        936        308   800
               2     3       07:02:00     07:22:00    M  B        461        252   200
               3     4       07:04:00     08:03:00    M  J        428         10  1000
               4     5       07:06:00     08:13:00    M  F        449        124   600


In [3]:
### 
### CREATING THE MODEL
###

##
## Initial values:
##
train_type = [
    "OC", 
    "OH"
    ]
cost_train_type = {
    "OC": 260,
    "OH": 210
}
length_train_type = {
    "OC": 100, 
    "OH": 70
    }
capacity_train_type = {
    "OC": 620, 
    "OH": 420
    }

#
# Data manipulated initial values:
#
cross_section = set(
    df['From'].values + df['To'].values
    ) #NOTE: Not used

trips_using_line400 = ((df.loc[df['line_400'] == 1]).index + 1).to_list() # +1 because the indexes start with 0, but the trips at 1. 


###
### Creation of the Model:
###
def create_general_model():
    model = pyo.ConcreteModel()

    ##
    ## CREATION OF SETS
    ##
    model.trips = pyo.Set(
        initialize = df['Trip']
        )

    model.train_type = pyo.Set(
        initialize = train_type
        )

    ##
    ## CREATION OF PARAMETERS
    ##
    model.cost_train_type = pyo.Param(
        model.train_type, 
        initialize = cost_train_type
    )

    model.length_train_type = pyo.Param(
        model.train_type, 
        initialize = length_train_type
        )

    model.capacity_train_type = pyo.Param(
        model.train_type, 
        initialize = capacity_train_type
        )
    
    #
    # Parameters created using functions:
    #
    model.trips_using_line400 = pyo.Param(
        model.trips,  # Assuming model.cross_section contains all possible cross sections
        initialize={trip: 1 if trip in trips_using_line400 else 0 for trip in model.trips}
    )

    model.passengers_per_trip = pyo.Param(
        model.trips, 
        initialize = {trip: df['Demand(μ)'].loc[trip - 1] for trip in model.trips} # -1 because the trips start with trip 1, but the df with index 0. So, loc[200] results in an error, because there are only 199 records. 
    )

    return model


model = create_general_model()


In [4]:
###
### ADDING SPECIFICS FOR MODEL1
###

##
## Variable specific for Model1
##
def specific_variables_Model1(model):
    #
    # Index set for allocation variable model1
    #
    model.index_set_allocation = pyo.Set(
        initialize = model.trips * model.train_type
        )

    #
    # Variable for allocation in Model1
    #
    # the variable represents the number of trains per type allocated to a trip.
    #
    model.allocation_train_numbers = pyo.Var(
        model.index_set_allocation, 
        domain = pyo.NonNegativeIntegers, 
        name = 'train_allocation', 
        doc = 'The number of trains of a certain type allocated to a cross-section'
    )
    return model

##
## Constraints specific for Model1
##
def specific_constraints_Model1(model):
    #
    # Rule for restricting the combined train length. 
    #
    def rule_maximum_length(
            m, 
            trip
            ):
        combined_train_length = sum(
            m.length_train_type[(train_type)] * m.allocation_train_numbers[(trip, train_type)]
            for train_type in m.train_type
        )
        return combined_train_length <= 300 - 100 * m.trips_using_line400[(trip)]
    model.constr_maximum_length = pyo.Constraint(
        model.trips, 
        rule = rule_maximum_length
        )
    
    #
    # Rule for enforcing the ability to accomodate all passengers. 
    #
    def rule_passenger_limit(
            m, 
            trip
            ):
        available_capacity = sum(
            m.capacity_train_type[(train_type)] * m.allocation_train_numbers[(trip, train_type)]
            for train_type in m.train_type
        )
        return available_capacity >= int(m.passengers_per_trip[(trip)])
    model.constr_passenger_limit = pyo.Constraint(
        model.trips, 
        rule = rule_passenger_limit
        )
    
    #
    # Rules for limiting the 'favorate' train type to be max 0.25% higher than the less preferred. 
    #
    def rule_difference_between_number_of_train_types1(m):
        return 0.75 * sum(
                m.allocation_train_numbers[(trip, train_type)] 
                for trip in m.trips 
                for train_type in m.train_type if train_type == "OC"
            ) <= sum(
                m.allocation_train_numbers[(trip, train_type)] 
                for trip in m.trips 
                for train_type in m.train_type if train_type == "OH"
            )
    model.constr_difference_between_number_of_train_types1 = pyo.Constraint(
        rule=rule_difference_between_number_of_train_types1
    )

    def rule_difference_between_number_of_train_types2(m):
        return 0.75 * sum(
                m.allocation_train_numbers[(trip, train_type)] 
                for trip in m.trips 
                for train_type in m.train_type if train_type == "OH"
            ) <= sum(
                m.allocation_train_numbers[(trip, train_type)] 
                for trip in m.trips 
                for train_type in m.train_type if train_type == "OC"
            )
    model.constr_difference_between_number_of_train_types2 = pyo.Constraint(
        rule=rule_difference_between_number_of_train_types2
    )

    return model


##
## Objective rule specific for Model1
##
def specific_objective_Model1(model):
    #
    # Objective rule to minimize the total cost of all trains. 
    #
    def obj_minimize_total_cost(m):
        total_cost = sum(
            m.cost_train_type[(train_type)] * sum(
                m.allocation_train_numbers[(trip, train_type)] 
                for trip in m.trips
            )
            for train_type in m.train_type
        )
        return total_cost
    model.objective = pyo.Objective(
        rule=obj_minimize_total_cost,
        sense=pyo.minimize
    )
    return model


##
## Add all variables, constraints and objectives to Model1
##
model1 = specific_variables_Model1(model=model)
model1 = specific_constraints_Model1(model=model1)
model1 = specific_objective_Model1(model=model1)


In [5]:
###
### ADDING SPECIFICS FOR MODEL2
###


#
# Obtain all compositions
#
def get_compositions():
    # Define the lengths of each train type
    OH_length = 100
    OC_length = 70

    OH_cap = 420
    OC_cap = 620

    # Define the maximum length of a train
    max_train_length = 300

    # Generate all possible combinations of OH and OC trains
    all_combinations = {}

    for num_OH in range(max_train_length // OH_length + 1):
        for num_OC in range(max_train_length // OC_length + 1):

            length = num_OH * OH_length + num_OC * OC_length
            if length <= max_train_length:
                combination = []
                if num_OH > 0:  # Append 'OH' only if num_OH is greater than 0
                    combination.extend(['OH'] * num_OH)
                if num_OC > 0:  # Append 'OC' only if num_OC is greater than 0
                    combination.extend(['OC'] * num_OC)

                cap = num_OH * OH_cap + num_OC * OC_cap

                if length != 0:
                    all_combinations[str(combination)] = [length, cap, num_OC, num_OH]

    return all_combinations


##
## Parameter specific for Model2
##
def specific_parameter_Model2(model):
    #
    # Set containing all compositions
    #
    compositions = get_compositions()
    #ic(compositions)

    model.compositions = pyo.Set(initialize = compositions.keys())

    #
    # Set containing the index for the quantification of the number of train types per composition.
    #
    model.index_quantify_compositions = pyo.Set(
        initialize= model.compositions * model.train_type
    )

    #
    # Parameter indicating number of 'OC' and 'OH' within a composition
    #
    model.quantify_compositions = pyo.Param(
        model.index_quantify_compositions,
        initialize = {
            (composition, train_type): compositions[composition][2] if train_type == "OC" else compositions[composition][3]
            for composition in model.compositions for train_type in model.train_type
        }
    )

    return model


##
## Variable specific for Model2
##
def specific_variables_Model2(model):
    #
    # Index set for allocation variable model1
    #
    model.index_set_allocation = pyo.Set(initialize = model.trips * model.compositions)


    #
    # Variable for allocation in Model1
    #
    # the binary variable indicates the composition used on a trip.
    #
    model.allocation_compositions = pyo.Var(
        model.index_set_allocation,
        domain = pyo.Binary,
        name = "Specific composition on trip",
        doc = "Indicates which composition is used on a trip"
    )

    return model


##
## Constraints specific for Model2
##
def specific_constraints_Model2(model):

    #
    # Rule that ensures a trip can only have a single composition
    #
    def rule_one_composition_per_trip(m, trip):
        return sum(
            model.allocation_compositions[(trip, composition)]
            for composition in model.compositions
        ) == 1
    model.constr_one_composition_per_trip = pyo.Constraint(
        model.trips, 
        rule=rule_one_composition_per_trip
    )

    #
    # Rules for limiting the 'favorate' train type to be max 0.25% higher than the less preferred. 
    #
    def rule_difference_between_number_of_train_types1(model):
        return 0.75 * sum(
            model.quantify_compositions[(composition, "OH")] * model.allocation_compositions[(trip, composition)]
            for composition in model.compositions
            for trip in model.trips
        ) <= sum(
            model.quantify_compositions[(composition, "OC")] * model.allocation_compositions[(trip, composition)]
            for composition in model.compositions
            for trip in model.trips
        )
    model.constr_difference_between_number_of_train_types1 = pyo.Constraint(
        rule=rule_difference_between_number_of_train_types1
    )
    def rule_difference_between_number_of_train_types2(model):
        return 0.75 * sum(
            model.quantify_compositions[(composition, "OC")] * model.allocation_compositions[(trip, composition)]
            for composition in model.compositions
            for trip in model.trips
        ) <= sum(
            model.quantify_compositions[(composition, "OH")] * model.allocation_compositions[(trip, composition)]
            for composition in model.compositions
            for trip in model.trips
        )
    model.constr_difference_between_number_of_train_types2 = pyo.Constraint(
        rule=rule_difference_between_number_of_train_types2
    )

    return model


def specific_objective_Model2(model):
    def obj_minimize_total_cost(model):
        return model.cost_train_type["OH"] * sum(
                model.quantify_compositions[(composition, "OH")] * model.allocation_compositions[(trip, composition)]
                for composition in model.compositions
                for trip in model.trips
            ) + model.cost_train_type["OC"] * sum(
                model.quantify_compositions[(composition, "OC")] * model.allocation_compositions[(trip, composition)]
                for composition in model.compositions
                for trip in model.trips
            )
    model.objective = pyo.Objective(rule=obj_minimize_total_cost, sense=pyo.minimize)

    return model

model2 = specific_parameter_Model2(model=model)
model2 = specific_variables_Model2(model=model2)
model2 = specific_constraints_Model2(model=model2)
model2 = specific_objective_Model2(model=model2)

(type=<class 'pyomo.core.base.set.OrderedScalarSet'>) on block unknown with a
new Component (type=<class 'pyomo.core.base.set.AbstractOrderedScalarSet'>).
This is usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


constr_difference_between_number_of_train_types1 (type=<class
'pyomo.core.base.constraint.ScalarConstraint'>) on block unknown with a new
Component (type=<class
'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
constr_difference_between_number_of_train_types2 (type=<class
'pyomo.core.base.constraint.ScalarConstraint'>) on block unknown with a new
Component (type=<class
'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.objective.ScalarObjective'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.objective.ScalarObjective'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


In [6]:
###
### CREATING THE SOLVING STRUCTURE
###


##
## Selecting the solver
##
solver = pyo.SolverFactory('cbc')


##
## Solve a specific model
##
def solve_model(model_to_solve, time_limit: int = 90, print_info: bool = True):
    #
    # Solve the model
    #
    results = solver.solve(
        model_to_solve, 
        options={'seconds': time_limit}
        )#, tee=True)

    #
    # Check the solver status
    #
    if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
        print("Solver terminated successfully. Model is feasible.")
    elif results.solver.termination_condition == pyo.TerminationCondition.infeasible:
        print("Solver terminated: Model is infeasible.")
    else:
        print("Solver terminated with non-optimal solution.")

    #
    # Output the information from the solver if necessary. 
    #
    if print_info:
        ic(results)
        ic(results.solver.time)
    
    return results


##
## Obtain solution from Model1
##
def get_solution_df_Model1(model, display_solution_df: bool = False):
    #
    # Use a dictionary to store all information from the decision variable
    #
    solution_dict = {}

    for trip in model.trips:
        for train_type in model.train_type:
            solution_dict[(trip, train_type)] = model.allocation_train_numbers[(trip, train_type)].value

    #
    # Create and fill a pandas dataframe to store the data from the dictionairy in a visually pleasing way.
    #
    solution_DF = pd.DataFrame(index=model.trips, columns=model.train_type)

    for trip in model.trips:
        for train_type in model.train_type:
            solution_DF.loc[trip, train_type] = solution_dict.get((trip, train_type), np.nan)

    #
    # Display the solution dataframe
    #
    if display_solution_df:
        ic(solution_DF)
      
        
    return solution_DF


##
## Obtain solution from Model2
##
def get_solution_df_Model2(model, display_solution_df: bool = False):
    #
    # Use a dictionary to store all information from the decision variable
    #
    solution_dict = {}

    for trip in model.trips:
        for composition in model.compositions:
            solution_dict[(trip, composition)] = model.allocation_compositions[(trip, composition)].value

    #
    # Create and fill a pandas dataframe to store the data from the dictionairy in a visually pleasing way.
    #
    solution_DF = pd.DataFrame(index=model.trips, columns=model.compositions)

    for trip in model.trips:
        for composition in model.compositions:
            solution_DF.loc[trip, composition] = solution_dict.get((trip, composition), np.nan)

    #
    # Display the solution dataframe
    #
    if display_solution_df:
        ic(solution_DF)
      
        
    return solution_DF




In [7]:
###
### Solving the actual models
###

#
# Solving Model1
#
solve_model(model_to_solve=model1, time_limit=30)
solution1 = get_solution_df_Model1(model=model1, display_solution_df=True)


#
# Solving Model2
#
solve_model(model_to_solve=model2, time_limit=30)
solution2 = get_solution_df_Model2(model=model2, display_solution_df=True)

#FIXME: The solutions are not the same.

ic| results: {'Problem': [{'Name': 'unknown', 'Lower bound': 46300.0, 'Upper bound': 46300.0, 'Number of objectives': 1, 'Number of constraints': 602, 'Number of variables': 2400, 'Number of binary variables': 2000, 'Number of integer variables': 2400, 'Number of nonzeros': 2000, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 1.3, 'Wallclock time': 1.37, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 50, 'Number of created subproblems': 50}, 'Black box': {'Number of iterations': 56}}, 'Error rc': 0, 'Time': 1.4007079601287842}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}
ic| results.solver.time: 1.4007079601287842
ic| solution_DF:       OC   OH
                 1    3.0  0.0
                 2    3.0  0.0
                 

Solver terminated successfully. Model is feasible.


ic| results: {'Problem': [{'Name': 'unknown', 'Lower bound': 46300.0, 'Upper bound': 46300.0, 'Number of objectives': 1, 'Number of constraints': 602, 'Number of variables': 2400, 'Number of binary variables': 2000, 'Number of integer variables': 2400, 'Number of nonzeros': 2000, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 1.32, 'Wallclock time': 1.37, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 50, 'Number of created subproblems': 50}, 'Black box': {'Number of iterations': 56}}, 'Error rc': 0, 'Time': 1.3868160247802734}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}
ic| results.solver.time: 1.3868160247802734
ic| solution_DF:     ['OC'] ['OC', 'OC'] ['OC', 'OC', 'OC'] ['OC', 'OC', 'OC', 'OC'] ['OH']  \
              

Solver terminated successfully. Model is feasible.


3                  0.0  
                 4                  0.0  
                 5                  0.0  
                 ..                 ...  
                 196                0.0  
                 197                0.0  
                 198                0.0  
                 199                0.0  
                 200                0.0  
                 
                 [200 rows x 10 columns]
